In [1]:
#5.1
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1621,  0.3344, -0.0975,  0.1109,  0.1315, -0.4677,  0.2131, -0.0483,
          0.2338, -0.2709],
        [-0.1159,  0.4168, -0.2490,  0.1038,  0.1473, -0.4194,  0.0854, -0.0716,
          0.1894, -0.2436]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.0099,  0.1454,  0.2838, -0.3136,  0.0943, -0.1703, -0.0059,  0.1608,
          0.0474, -0.3629],
        [-0.0375,  0.1417,  0.1619, -0.2816,  0.0546, -0.0868,  0.0751,  0.0728,
          0.2317, -0.3503]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1283, -0.2505,  0.0692, -0.0353,  0.0963, -0.1223, -0.0376, -0.2937,
          0.1487,  0.2415],
        [-0.1198, -0.2287,  0.0902, -0.1079,  0.1324, -0.0883, -0.0382, -0.2606,
          0.1839,  0.2386]], grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.0785, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0739, grad_fn=<SumBackward0>)

In [1]:
#5.2

In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3335],
        [-0.4271]], grad_fn=<AddmmBackward0>)

In [3]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1782, -0.2035, -0.2761, -0.2026, -0.1256, -0.0182,  0.1783,  0.2404]])), ('bias', tensor([-0.2701]))])


In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2701], requires_grad=True)
tensor([-0.2701])


In [5]:
net[2].weight.grad == None

True

In [6]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
print(*[1])

1


In [8]:
net.state_dict()['2.bias'].data

tensor([-0.2701])

In [9]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4647],
        [-0.4647]], grad_fn=<AddmmBackward0>)

In [10]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [11]:
rgnet[0][1][0].bias.data

tensor([-0.1368, -0.4360,  0.2891,  0.1783, -0.4031,  0.1444, -0.0729,  0.1658])

In [12]:
rgnet[0][1][0].state_dict()

OrderedDict([('weight',
              tensor([[ 0.3309, -0.2018, -0.4697, -0.4279],
                      [-0.3322, -0.1260,  0.4053,  0.3037],
                      [-0.0870,  0.1222, -0.1533,  0.4208],
                      [ 0.2295,  0.2351,  0.3565, -0.3006],
                      [ 0.1078,  0.0512,  0.3338, -0.4425],
                      [ 0.0898, -0.0208, -0.1370,  0.3823],
                      [ 0.3682, -0.1217, -0.3703,  0.0986],
                      [ 0.1293,  0.0053,  0.0513, -0.3592]])),
             ('bias',
              tensor([-0.1368, -0.4360,  0.2891,  0.1783, -0.4031,  0.1444, -0.0729,  0.1658]))])

In [13]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([0.0142, 0.0073, 0.0066, 0.0026]), tensor(0.))

In [14]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [15]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.3932, 0.1772, 0.6713, 0.6673])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [16]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.5386, -9.8476,  0.0000,  9.8381],
        [-0.0000,  0.0000,  7.8190,  5.3683]], grad_fn=<SliceBackward0>)

In [17]:
a=3

In [18]:
a*= a>10

In [19]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [20]:
net[0].weight[:]

tensor([[-9.5386, -9.8476,  0.0000,  9.8381],
        [-0.0000,  0.0000,  7.8190,  5.3683],
        [ 6.7490,  0.0000,  0.0000, -0.0000],
        [ 5.0517, -9.7502,  6.4220, -0.0000],
        [ 9.5229, -9.5778, -5.8290,  0.0000],
        [ 0.0000, -0.0000,  0.0000, -0.0000],
        [ 5.9020, -0.0000, -0.0000, -5.0403],
        [-5.1756,  0.0000,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [21]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-9.5386, -9.8476,  0.0000,  9.8381],
                      [-0.0000,  0.0000,  7.8190,  5.3683],
                      [ 6.7490,  0.0000,  0.0000, -0.0000],
                      [ 5.0517, -9.7502,  6.4220, -0.0000],
                      [ 9.5229, -9.5778, -5.8290,  0.0000],
                      [ 0.0000, -0.0000,  0.0000, -0.0000],
                      [ 5.9020, -0.0000, -0.0000, -5.0403],
                      [-5.1756,  0.0000,  0.0000,  0.0000]])),
             ('0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0.])),
             ('2.weight',
              tensor([[-9.6828, -0.0000, -5.2050, -0.0000,  5.3223,  9.2863, -0.0000, -7.0476]])),
             ('2.bias', tensor([0.]))])

In [22]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -8.8476,  1.0000, 10.8381])

In [23]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [1]:
#5.3

In [2]:
import torch
from torch import nn
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(),nn.Linear(256,10))
print(net)

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


/root/anaconda3/envs/d2l/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [3]:
[net[i].state_dict() for i in range(len(net))]

[OrderedDict([('weight', <UninitializedParameter>),
              ('bias', <UninitializedParameter>)]),
 OrderedDict(),
 OrderedDict([('weight',
               tensor([[-1.7882e-02, -5.9462e-02, -6.3973e-03,  ..., -3.5650e-02,
                        -2.1644e-05,  3.3874e-02],
                       [ 5.9455e-02, -4.3125e-02, -3.4836e-02,  ..., -7.2742e-03,
                        -2.1134e-02,  6.0765e-02],
                       [-3.6757e-02,  3.1354e-02, -5.0320e-02,  ..., -2.7340e-02,
                         5.8414e-02,  3.4922e-02],
                       ...,
                       [ 6.0380e-02,  5.2377e-02,  4.5505e-02,  ..., -1.1676e-02,
                        -1.4830e-02, -1.9998e-02],
                       [-5.4820e-02, -4.2243e-02, -4.0840e-02,  ..., -5.8522e-04,
                         4.8966e-02, -5.7866e-02],
                       [ 9.1680e-03,  5.3936e-02,  4.3567e-02,  ..., -2.0278e-02,
                         2.1227e-02, -2.2285e-02]])),
              ('bias',
   

In [4]:
low = torch.finfo(torch.float32).min/10
high = torch.finfo(torch.float32).max/10
X = torch.zeros([2,20],dtype=torch.float32).uniform_(low, high)
net(X)
print(net)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [5]:
low

-3.4028234663852886e+37

In [6]:
high

3.4028234663852886e+37

In [7]:
#5.4

In [8]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [9]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [10]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [11]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(3.0268e-09, grad_fn=<MeanBackward0>)

In [12]:
Y

tensor([[-4.4269e-02,  1.5388e-01, -2.8648e-01,  3.7712e-01, -3.1574e-01,
         -8.8463e-01, -1.7592e-01,  5.5223e-01,  1.9301e-01, -5.5548e-01,
         -3.3577e-01, -7.6941e-02,  2.5568e-01, -2.6437e-01, -2.5563e-01,
         -2.4424e-01,  1.1911e-01,  3.5435e-01,  6.4722e-02,  4.0116e-01,
         -2.8704e-01, -1.2416e-01, -1.1496e-01, -3.9990e-01,  2.2693e-01,
          1.5481e-01, -3.8928e-01, -2.0838e-01,  2.9340e-02,  4.4121e-02,
          5.4471e-01, -2.5810e-01, -5.9561e-01,  1.9464e-02, -2.1710e-01,
         -2.2675e-01,  9.8288e-01,  6.4091e-01,  2.3451e-01,  2.2695e-02,
          1.9574e-01, -2.9510e-02, -1.6844e-01,  3.2973e-01, -2.2090e-01,
         -2.2187e-01,  3.4128e-01, -6.9265e-01, -2.5459e-01, -1.4466e-01,
          8.8989e-02,  1.2924e-01, -2.4259e-02, -2.1263e-01,  2.0965e-02,
          1.9710e-01, -3.5008e-01,  3.5457e-01,  1.6156e-02, -3.0908e-01,
         -1.4371e-01,  4.0964e-01,  3.0087e-01,  1.2807e-01,  2.2406e-01,
         -1.6790e-01,  5.1771e-01,  2.

In [13]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [14]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.8083, -0.1043, -0.2994],
        [ 1.7996,  1.9506,  0.1189],
        [ 1.3044, -1.0015, -0.3298],
        [-1.0542,  0.0592, -0.2689],
        [ 1.3766, -1.7915,  0.8468]], requires_grad=True)

In [15]:
linear(torch.rand(2, 5))

tensor([[3.0489, 0.8498, 0.0000],
        [1.6331, 1.5277, 0.0000]])

In [16]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

In [20]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 4.3876e-01, -1.2605e+00, -5.9349e-01,  7.5111e-01, -1.1380e+00,
                        2.5091e+00, -1.4615e+00,  7.7578e-02],
                      [ 3.6788e-01, -1.1163e+00, -1.7078e-02, -9.2126e-02, -4.6682e-01,
                        7.5107e-01,  2.9649e-01, -1.7432e+00],
                      [-2.0523e+00, -5.6545e-01, -1.3460e+00, -1.5920e-01, -2.5378e-02,
                        6.7565e-01, -1.8494e+00, -1.1006e+00],
                      [-8.5919e-01,  6.9389e-01,  2.4644e-02,  2.4950e-01, -1.1889e-01,
                        3.2257e-01, -9.8427e-01, -2.9167e-01],
                      [ 1.3556e-01, -1.5844e+00,  7.3688e-01,  4.4880e-01, -1.4792e-01,
                        8.5940e-01, -2.2937e+00, -1.3677e+00],
                      [ 6.3660e-01, -7.4961e-01, -1.2010e+00, -8.8958e-01,  1.6995e+00,
                       -1.4833e+00, -6.1816e-01,  6.0311e-01],
                      [-9.6221e-01,  1.6177e+00,  5.9987e-02,  1.091

In [22]:
#5.5

In [24]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [25]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [26]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [27]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [28]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [29]:
torch.save(net.state_dict(), 'mlp.params')

In [35]:
net.hidden.weight

Parameter containing:
tensor([[ 0.1208, -0.0664,  0.0678,  ...,  0.1392,  0.1818,  0.0259],
        [-0.0975, -0.1599,  0.0427,  ..., -0.1208, -0.0690,  0.2153],
        [ 0.1877,  0.0040,  0.1067,  ..., -0.1955, -0.1865, -0.1623],
        ...,
        [ 0.0304, -0.1066, -0.1152,  ..., -0.0456,  0.0459,  0.1431],
        [-0.1897, -0.1595, -0.1271,  ..., -0.2012, -0.0152, -0.0188],
        [ 0.0437, -0.1934,  0.1878,  ...,  0.1919, -0.1077, -0.0295]],
       requires_grad=True)

In [33]:
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.1208, -0.0664,  0.0678,  ...,  0.1392,  0.1818,  0.0259],
                      [-0.0975, -0.1599,  0.0427,  ..., -0.1208, -0.0690,  0.2153],
                      [ 0.1877,  0.0040,  0.1067,  ..., -0.1955, -0.1865, -0.1623],
                      ...,
                      [ 0.0304, -0.1066, -0.1152,  ..., -0.0456,  0.0459,  0.1431],
                      [-0.1897, -0.1595, -0.1271,  ..., -0.2012, -0.0152, -0.0188],
                      [ 0.0437, -0.1934,  0.1878,  ...,  0.1919, -0.1077, -0.0295]])),
             ('hidden.bias',
              tensor([-0.1846, -0.1519, -0.1118,  0.1135, -0.0456,  0.2072,  0.1796, -0.0482,
                      -0.0389, -0.1063,  0.0293, -0.1453, -0.0616,  0.0753, -0.1940,  0.2008,
                       0.2004, -0.1504,  0.1493,  0.0775, -0.0213, -0.1020, -0.1384, -0.1456,
                      -0.0165,  0.1927, -0.1766,  0.0562, -0.0115, -0.1108,  0.0278, -0.1705,
                       0.2003,

In [36]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [37]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [39]:
#5.6

In [40]:
!nvidia-smi

modprobe: ERROR: could not insert 'nvidia': No such device
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [41]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [42]:
torch.device('cpu')
torch.device('cuda')
torch.device('cuda:1')

device(type='cuda', index=1)

In [43]:
torch.cuda.device_count()

/root/anaconda3/envs/d2l/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


0

In [44]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

In [45]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [49]:
month=list(range(1,12+1))

In [50]:
month

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [51]:
day=list(range(1,6+1))

In [52]:
day

[1, 2, 3, 4, 5, 6]

In [65]:
for i,j in zip(month,day):
    print(f'today is {i:10.2f} month {j} day')

today is       1.00 month 1 day
today is       2.00 month 2 day
today is       3.00 month 3 day
today is       4.00 month 4 day
today is       5.00 month 5 day
today is       6.00 month 6 day


In [54]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [68]:
f'a{i}'=123

SyntaxError: cannot assign to f-string expression (3279034785.py, line 1)

In [69]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [70]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.0222, 0.8680, 0.2273],
        [0.0805, 0.9940, 0.7173]])

In [ ]:
Z = X.cuda(1)
print(X)
print(Z)

In [ ]:
Y + Z

In [ ]:
Z.cuda(1) is Z

In [ ]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [ ]:
net(X)

In [ ]:
net[0].weight.data.device